In [ ]:
Copyright (c) 2024, ETH Zurich

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt 
import os
import scipy.stats as stats
import scipy.ndimage
from tqdm import tqdm
import spekpy as spk
import h5py

from scipy import interpolate
import json

In [ ]:
rave_sim_dir = Path('path/to/rave-sim')
simulations_dir = Path('path/to/data/output')
scratch_dir = simulations_dir

sys.path.insert(0, str(rave_sim_dir / "nist_lookup"))
from nist_lookup.xraydb_plugin import xray_delta_beta

In [ ]:
sys.path.insert(0, str(rave_sim_dir / "big-wave"))
import multisim
import config
import util
import propagation


In [ ]:
# matplotlib style
plt.style.use("default")

# set FIGWIDTH to latex's \textwidth
FIGWIDTH = 3
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 6
plt.rcParams["figure.figsize"] = (FIGWIDTH, FIGWIDTH * 2 / 3)
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.constrained_layout.use"] = "True"

# images
plt.rcParams["image.interpolation"] = "bicubic"
plt.rcParams["image.cmap"] = "Greys_r"

# axes
# plt.rcParams["axes.spines.right"] = False
# plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.edgecolor"] = "0.7"
plt.rcParams["axes.linewidth"] = "1"

# legend
plt.rcParams["legend.frameon"] = False

plt.rcParams["lines.markersize"] = 3
# plt.rcParams["lines.markerfacecolor"] = "white"
# Okabe-Ito palette
plt.rcParams["axes.prop_cycle"] = plt.cycler(
    color=[
        "#000000",
        "#E69F00",
        "#56B4E9",
        "#009E73",
        "#F0E442",
        "#0072B2",
        "#D55E00",
        "#CC79A7",
    ],
    marker=["o", "^", "s", "p", "D", "v", "v", "d"],
)

plt.rcParams['axes.grid'] = False

In [ ]:
def calculate_G1_height(eng):
    # constants
    h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
    c_0 = 299792458 # speed of light in m / s
    eV_to_joule = 1.602176634*10**(-19)
    N_A = 6.02214086 * 10**23 #[1/mol]
    
    lambda_ = h * c_0 / (eng*eV_to_joule)
    delta_diff = xray_delta_beta('Au', 19.32, eng)[0] -  xray_delta_beta('Si', 2.34, eng)[0]
    height = np.pi  * lambda_ / (2*np.pi * delta_diff)

    return height


def signal_retrieval_least_squares(data, period=None, axis=-1):
    if axis != -1:
        data = np.moveaxis(data, axis, -1)

    nsteps = data.shape[-1]

    if period is None:
        period = nsteps

    phi = np.linspace(0, 2 * np.pi * nsteps / (period), nsteps, endpoint=False)
    M = np.c_[np.sin(phi), np.cos(phi), np.ones(nsteps)]
    res, chi2, _rank, _sing_vals = np.linalg.lstsq(M, data.reshape((-1, nsteps)).T, rcond=-1)

    res = res.T.reshape((*data.shape[:-1], -1))

    dabs = res[...,2]
    dphase = -np.arctan2(res[...,0], res[...,1])
    dvis = np.sqrt(res[...,0]**2 + res[...,1]**2) / dabs

    # normalization to the total number of counts
    dabs *= nsteps

    return dabs, dphase, dvis, np.nanmean(chi2)


def calculate_pixel_intensity(x, fringe, pxEdges, statistics = 'sum'):
    fringeStats = stats.binned_statistic(x, fringe, bins=pxEdges, statistic = statistics)
    return fringeStats.statistic

def perform_binned_signal_retrieval(x, wf, pxSize, nrSteps, plot_curve = True):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = []
    for i in range(nrSteps):
        int_px.append(calculate_pixel_intensity(x, wf[i,:], pxEdges))
    int_px = np.asarray(int_px)

    trans, phase, vis, _ = signal_retrieval_least_squares(int_px, period = nrSteps, axis = 0)
    if plot_curve:
        plot_curves(trans, phase, vis, pxEdges)
    return int_px, trans, phase, vis, pxEdges

def plot_curves(trans, phase, vis, pxEdges):

    fig, axs = plt.subplots(figsize=(15,6), sharex = True, nrows = 1, ncols = 2)
    axs[0].plot(pxEdges[1:], trans, label = 'Transmission')
    axs[0].set_title('Transmission')
    axs[1].plot(pxEdges[1:], vis, label = 'Visibility')
    axs[1].set_title('Visibility')

def perform_binning(x, wf, pxSize):
    leftSide = np.arange(0-pxSize/2, np.min(x), -pxSize)
    rightSide = np.arange(0 + pxSize/2, np.max(x), pxSize)
    pxEdges = np.concatenate([np.flip(leftSide), rightSide])
    int_px = calculate_pixel_intensity(x, wf, pxEdges, statistics = 'mean')
    return int_px


    #np.random.seed(1)
    
    sample_energy = np.random.choice(x, p=y)
    if sample_energy >= low_threshold:
        return True
    else:
        return False
    
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def calculate_SNR(wavefronts, detector_x, phase_steps):
    summed_wf = np.zeros_like(wavefronts[0][0])
        
    for point in wavefronts:
        wf, x_point, eng = point
        
        summed_wf += wf

    return summed_wf

def get_subdict(dict_, idx):
    sub_dict = {}
    for key in dict_.keys():
        sub_dict[key] = dict_[key][idx]
    return sub_dict

def calc_Vis_theoretical(eng, Edes,m):
    V = 2/np.pi * np.abs(np.sin(np.pi / 2 * Edes / eng)**2 * np.sin(m * np.pi / 2 * Edes / eng))
    return V

def mu_h2o(eng):
    lambda_ = h * c_0 / (eng*eV_to_joule)
    beta = xray_delta_beta('H2O', 1.0, eng)[1]
    return 4 * np.pi / lambda_ * beta

In [ ]:
# constants
h = 6.62607004 * 10**(-34) # planck constant in mˆ2 kg / s
c_0 = 299792458 # speed of light in m / s
eV_to_joule = 1.602176634*10**(-19)
N_A = 6.02214086 * 10**23 #[1/mol]
E_des = 46000

lambda_ = h * c_0 / (E_des*eV_to_joule)
p2 = 4.2*10**(-6)
p0 = p1 = p2

Dn_5 = 5*p2**2/(2*lambda_) / 2

z_g0 = 0.1
z_g1 = z_g0 + Dn_5
z_g2 = z_g0 + 2*Dn_5
z_detector = z_g2 + 0.01

h0 = h2 = 180e-6
h1 = 59e-6

print("Z0: ", z_g0)
print("Z1: ", z_g1)
print("Z2: ", z_g2)
print("Z Detector: ", z_detector)

print(Dn_5)

In [ ]:
N = 2**26
max_energy = 70000
dx = propagation.max_dx(z_g0, 200e-6, N, propagation.convert_energy_wavelength(max_energy))

In [ ]:
s = spk.Spek(kvp=70, dk = 0.1, th = 10) # Create a spectrum
s.multi_filter((('Be', 0.15), ('Al', 3))) # Create a spectrum
k, f = s.get_spectrum(edges=True,diff=True, flu=True ) # Get the spectrum

energyRange = [4000, 70000]
dE = 100
filtering = 0.000

energies = np.arange(5, 70+0.1, 0.1)*1e3


tube_spectrum_txt = interpolate.interp1d(k*1e3, f, fill_value = 'extrapolate')
spec_txt = tube_spectrum_txt(energies)

with h5py.File('spectrum_70_spekpy_filtered_3mmAl.h5', 'w') as h5:
    h5.create_dataset('pdf', data =  spec_txt/ np.sum(spec_txt))
    h5.create_dataset('energy', data = energies) 

In [ ]:
plt.plot(energies, spec_txt)

In [ ]:
h0

In [ ]:
N = 2**26
max_energy = 70000
dx = propagation.max_dx(z_g0, 150e-6, N, propagation.convert_energy_wavelength(max_energy))

In [ ]:



config_dict = {
            "sim_params": {
                "N": N,
                "dx": dx,
                "z_detector": z_g2 + 500e-6,
                "detector_size": 0.004,
                "detector_pixel_size_x": 1e-7,
                "detector_pixel_size_y": 1.0,
                "chunk_size": 256 * 1024 * 1024 // 16,  # use 256MB chunks
            },
            "use_disk_vector": False,
            "save_final_u_vectors": False,
            "dtype": "c8",
            "multisource": {
                "type": "points",
                "energy_range": [11000, 70000],
                "x_range": [-150e-6, 150e-6],
                "z": 0.0,
                "nr_source_points": 1200,
                "seed": 1,
                "spectrum": "/path_to_spectrum/spectrum_70_spekpy_filtered_3mmAl.h5",
            },
            "elements": [
                {
                    "type": "grating",
                    "pitch": 4.2*1e-6,
                    "dc": [0.5, 0.5],
                    "z_start": z_g0,
                    "thickness": h0,
                    "nr_steps": 30,
                    "x_positions": [0.0],
                    "substrate_thickness": 500e-6 - h0,
                    "mat_a": ["C5H8O2", 1.19],
                    "mat_b": ["Au", 19.32],
                    "mat_substrate": ["C", 2.26],
                },           
                {
                    "type": "grating",
                    "pitch": p1,
                    "dc": [0.5, 0.5],
                    "z_start": z_g1,
                    "thickness": h1,
                    "nr_steps": 10,
                    "x_positions": [0.0],
                    "substrate_thickness": 200 * 1e-6 - h1,
                    "mat_a": ["Si", 2.34],
                    "mat_b": None,
                    "mat_substrate": ["Si", 2.34],
                },
                {
                    "type": "grating",
                    "pitch": p2,
                    "dc": [0.5, 0.5],
                    "z_start": z_g2,
                    "thickness": h2,
                    "nr_steps": 30,
                    "x_positions": (np.arange(5) * p2/5).tolist(),
                    "substrate_thickness": 500*1e-6 - h2,
                    "mat_a": ["C5H8O2", 1.19],
                    "mat_b": ["Au", 19.32],
                    "mat_substrate": ["C", 2.26],
                },
            ],
    }
sim_path = multisim.setup_simulation(config_dict, Path("."), simulations_dir)

for i in tqdm(range(config_dict["multisource"]["nr_source_points"])):
    os.system(f"CUDA_VISIBLE_DEVICES=0 path/to/rave-sim/fast-wave/build-Release/fastwave -s {i} {sim_path}")

In [ ]:
energy_thresholds = np.arange(34000, 70000+2000, 2000)

# Disclaimer detector response 
For the following code lines a detector response is required to generate the same plots as in: Simulation Framework for X-ray Grating Interferometry Optimization

The files for the detector response are not included but can be generated using DOI: <a href="https://aapm.onlinelibrary.wiley.com/doi/10.1002/mp.12863">10.1002/mp.12863</a>

The package can be requested on the following link: <a href="https://pctk.jhu.edu/">https://pctk.jhu.edu/</a>

For the first detector response: function_name – interpolate_response_function_mat_10 the following parameters were used:
- $r_0 = 10$
- $\sigma_e = 0.5\,keV$
- $\Delta_{pix} = 100 \,\mu m$
- $dz = 750\,\mu m$
- $\rho_{PCD} = 5.85\,g/cm^2$
- $NI = 1$
- $E_{th1} = 11 keV$

For the second detector response: function_name – interpolate_response_function_mat_20 the following parameters were used:

- $r_0 = 20$
- $\sigma_e = 2\,keV$
- $\Delta_{pix} = 100 \,\mu m$
- $dz = 750\,\mu m$
- $\rho_{PCD} = 5.85\,g/cm^2$
- $NI = 1$
- $E_{th1} = 11 keV$

In [ ]:
def interpolate_response_function_mat_10(energy, low_threshold, upper_threshold):
    srf_mat = scipy.io.loadmat('path_to_detector_response/SRF_model_10_0_5s_1TH.mat')['SRF_all'] # File with detector response 
    energies_mat = np.arange(0, srf_mat.shape[0])*1e3 - 1000
    #low_threshold = int(low_threshold / 1e3)
    #upper_threshold = int(upper_threshold / 1e3)
    energy = int(energy / 1e3)
    y = srf_mat[1:, energy]
    x = energies_mat[1:]
    y = y / np.sum(y)

    idx = find_nearest(x, low_threshold)
    idx_2 = find_nearest(x, upper_threshold)

    return np.sum(y[idx:idx_2])

def interpolate_response_function_mat_20(energy, low_threshold, upper_threshold):
    srf_mat = scipy.io.loadmat('path_to_detector_response/SRF_model_20_2s_1TH.mat')['SRF_all'] # File with detector response
    energies_mat = np.arange(0, srf_mat.shape[0])*1e3 - 1000
    #low_threshold = int(low_threshold / 1e3)
    #upper_threshold = int(upper_threshold / 1e3)
    energy = int(energy / 1e3)
    y = srf_mat[1:, energy]
    x = energies_mat[1:]
    y = y / np.sum(y)

    idx = find_nearest(x, low_threshold)
    idx_2 = find_nearest(x, upper_threshold)

    return np.sum(y[idx:idx_2])

In [ ]:


#### Without detector response
vis_all_sim = []
int_all_sim = []
for upper_eng in energy_thresholds:


    pxSize = 100e-6
    config_dict = config.load(Path(str(sim_path) + '/config.yaml'))
    sp = config_dict["sim_params"]
    detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
    pixel_rectangle = np.abs(detector_x) <= pxSize
    
    wavefronts_ref = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[11000, upper_eng])
    wf_ref = np.zeros_like(wavefronts_ref[0][0])
    
    energies = []
    for wave in wavefronts_ref:
        energies.append(wave[2])
        wf_ref += wave[0] 
    
    phase_steps = 5
    del wavefronts_ref
    
    convolved_ref = []
    for i in range(phase_steps):
        convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))
    
    convolved_ref = np.asarray(convolved_ref)
    
    int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
    int_px, trans_orig, phase_orig, vis_orig, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)

    vis_all_sim.append(vis_orig)
    int_all_sim.append(trans_ref)

vis_all_sim = np.asarray(vis_all_sim)
int_all_sim = np.asarray(int_all_sim)

sens_all = vis_all_sim * np.sqrt(int_all_sim)
sens_normalized_180 = np.mean(sens_all[:,15:23], axis = 1)/np.max(np.mean(sens_all[:,15:23], axis = 1))
vis_meaned = np.mean(vis_all_sim[:, 15:23], axis = 1)


#### Detector response 1
vis_all_sim = []
int_all_sim = []
for upper_eng in energy_thresholds:
    for s_path in all_simulations:
        sim_path = Path(os.path.join(simulations_dir, '2024', '05', s_path))
        pxSize = 100e-6
        config_dict = config.load(Path(str(sim_path) + '/config.yaml'))
        sp = config_dict["sim_params"]
        detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
        pixel_rectangle = np.abs(detector_x) <= pxSize
        
        wavefronts_ref = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[20000, 70000])
        wf_ref = np.zeros_like(wavefronts_ref[0][0])
        
        energies = []
        for wave in wavefronts_ref:
            energies.append(wave[2])
            wf_ref += wave[0] * interpolate_response_function_mat_20(wave[2], 20000, upper_eng)
        
        phase_steps = 5
        del wavefronts_ref
        
        convolved_ref = []
        for i in range(phase_steps):
            convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))
        
        convolved_ref = np.asarray(convolved_ref)
        
        int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
        int_px, trans_orig, phase_orig, vis_orig, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)

        vis_all_sim.append(vis_orig)
        int_all_sim.append(trans_ref)

vis_all_sim = np.asarray(vis_all_sim)
int_all_sim = np.asarray(int_all_sim)

sens_all = vis_all_sim * np.sqrt(int_all_sim)
sens_normalized_180_dec_response_20 = np.mean(sens_all[:,15:23], axis = 1)/np.max(np.mean(sens_all[:,15:23], axis = 1))
vis_meaned = np.mean(vis_all_sim[:, 15:23], axis = 1)



#### Detector response 2
vis_all_sim = []
int_all_sim = []
for upper_eng in energy_thresholds:
    for s_path in all_simulations:
        sim_path = Path(os.path.join(simulations_dir, '2024', '05', s_path))
        pxSize = 100e-6
        config_dict = config.load(Path(str(sim_path) + '/config.yaml'))
        sp = config_dict["sim_params"]
        detector_x = util.detector_x_vector(sp["detector_size"], sp["detector_pixel_size_x"])
        pixel_rectangle = np.abs(detector_x) <= pxSize
        
        wavefronts_ref = util.load_wavefronts_filtered(sim_path, x_range=None, energy_range=[20000, 70000])
        wf_ref = np.zeros_like(wavefronts_ref[0][0])
        
        energies = []
        for wave in wavefronts_ref:
            energies.append(wave[2])
            wf_ref += wave[0] * interpolate_response_function_mat_10(wave[2], 20000, upper_eng)
        
        phase_steps = 5
        del wavefronts_ref
        
        convolved_ref = []
        for i in range(phase_steps):
            convolved_ref.append(np.convolve(wf_ref[i,:], pixel_rectangle, mode = 'same'))
        
        convolved_ref = np.asarray(convolved_ref)
        
        int_px, trans_ref, phase_ref, vis_ref, pxEdges = perform_binned_signal_retrieval(detector_x, convolved_ref[:, :], pxSize, phase_steps, plot_curve = False)
        int_px, trans_orig, phase_orig, vis_orig, pxEdges = perform_binned_signal_retrieval(detector_x, wf_ref, pxSize, phase_steps, plot_curve = False)

        vis_all_sim.append(vis_orig)
        int_all_sim.append(trans_ref)

vis_all_sim = np.asarray(vis_all_sim)
int_all_sim = np.asarray(int_all_sim)

sens_all = vis_all_sim * np.sqrt(int_all_sim)
sens_normalized_180_dec_response_10 = np.mean(sens_all[:,15:23], axis = 1)/np.max(np.mean(sens_all[:,15:23], axis = 1))
vis_meaned = np.mean(vis_all_sim[:, 15:23], axis = 1)

In [ ]:
# matplotlib style
plt.style.use("default")

# set FIGWIDTH to latex's \textwidth
FIGWIDTH = 3
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 6
plt.rcParams["figure.figsize"] = (FIGWIDTH, FIGWIDTH * 2 / 3)
plt.rcParams["figure.dpi"] = 300
plt.rcParams["figure.constrained_layout.use"] = "True"

# images
plt.rcParams["image.interpolation"] = "bicubic"
plt.rcParams["image.cmap"] = "Greys_r"

# axes
# plt.rcParams["axes.spines.right"] = False
# plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.edgecolor"] = "0.7"
plt.rcParams["axes.linewidth"] = "0.8"

# legend
plt.rcParams["legend.frameon"] = True

plt.rcParams["lines.markersize"] = 3
# plt.rcParams["lines.markerfacecolor"] = "white"
# Okabe-Ito palette
plt.rcParams["axes.prop_cycle"] = plt.cycler(
    color=[
        "#000000",
        #"#56B4E9",
        "#009E73",
        "#F0E442",
        "#0072B2",
        "#D55E00",
        "#CC79A7",
    ],
    #marker=["o", "^", "s", "p", "D", "v", "v", "d"],
)


plt.rcParams['axes.grid'] = False

In [ ]:
mean_sens = np.load('Sensitivity_measured.npy')
error = np.load('Error_measured.npy')
energies_measurement = np.array([40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70])

In [ ]:
plt.figure(figsize=(3.3,2.5))
plt.plot(energies_measurement, mean_sens/np.max(mean_sens), label = r"Measured – $S_{max}$ at $E = 50keV$", marker = '^', linestyle = ":", linewidth = 1, markeredgecolor = 'black',  markeredgewidth = 0.2)


plt.errorbar(energies_measurement,  mean_sens/np.max(mean_sens), yerr=error/2, fmt='^', color='black',
                    ecolor='black', elinewidth=0.5, capsize=1)


#plt.errorbar(energies_measurement, sens_ideal/np.max(sens_ideal), yerr=error_bar/2/np.max(median_measurement), fmt='^', color='black',
#                    ecolor='black', elinewidth=0.5, capsize=1)

plt.plot(energy_thresholds/1e3, sens_normalized_180, label = r"Simulated – $S_{max}$ at $E = 52keV$", color = "#009E73")
plt.plot(energy_thresholds/1e3, sens_normalized_180_dec_response_10, color = "#0072B2")
plt.plot(energy_thresholds/1e3, sens_normalized_180_dec_response_20, label = r"Simulated with a detector " + "\n response $S_{max}$ at $E = 53$", color = "#0072B2")
plt.fill_between(energy_thresholds/1e3, sens_normalized_180_dec_response_10, sens_normalized_180_dec_response_20, alpha=0.2, color='#0072B2')


#plt.plot(energy_thresholds/1e3, sens_normalized_tap, label = 'Simulated Sensitivit tapering')

#plt.plot(energy_thresholds/1e3, sens_normalized, label = 'Simulated Sensitivit 170um')


#plt.plot(energies, sens_all, label = 'All energies')
plt.axvline(50, color = "#000000", linewidth = 0.8, linestyle = '--')
plt.axvline(54, color =  "#009E73", linewidth = 0.8, linestyle = '--')
plt.axvline(53, color =  "#0072B2", linewidth = 0.8, linestyle = '--')


plt.xlim(41, 70.5)
plt.legend()
#currentAxis = plt.gca()
#currentAxis.add_patch(Rectangle((50, 0.0), 20, 5000, alpha=0.3, color = 'red', hatch = '/', label = 'Ignored counts'))
#plt.legend(fontsize = 25)
#plt.grid('on')
plt.ylabel(r"Normalized Sensitivity $\sqrt{I} V$",  fontsize = 6)
plt.xlabel('Upper energy threshold detector in keV',  fontsize = 6)

plt.title('Sensitivity for 5th Talbot Order')
#plt.savefig('sensitivity_simulated_measured_with_dec_response_two_curves.png', bbox_inches = 'tight')